We are going to use Windows 10 system generated values to predict whether the system is under cyber attack or not !

# Required Libraries

In [1]:
import numpy as np
from numpy.ma.core import ceil
from scipy.spatial import distance #distance calculation
from sklearn.preprocessing import MinMaxScaler #normalisation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score #scoring
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib import animation, colors

# Loading the data and EDA


In [2]:
import pandas as pd
df = pd.read_csv('windows10_dataset.csv')
df.head()

,ts,Processor_DPC_Rate,Processor_pct_ Idle_Time,Processor_pct_ C3_Time,Processor_pct_ Interrupt_Time,Processor_pct_ C2_Time,Processor_pct_ User_Time,Processor_pct_ C1_Time,Processor_pct_ Processor_Time,Processor_C1_ransitions_sec,...,LogicalDisk(_Total) Avg Disk Write Queue Length,LogicalDisk(_Total) Avg Disk Queue Length,LogicalDisk(_Total) pct_ Disk Read Time,LogicalDisk(_Total) Disk Write Bytes sec,LogicalDisk(_Total) Disk Transfers sec,LogicalDisk(_Total) Avg Disk Bytes Transfer,LogicalDisk(_Total) pct_ Disk Write Time,LogicalDisk(_Total) Avg Disk sec Transfer,label,type
0,1554206309,4,29.90817156,0,0.078240397,0,61.02750947,29.90817156,66.2001486,480.0949912,...,0.134876119,0.163098103,2.822198461,1924988.236,402.5827836,9100.481592,13.48761188,0.000405118,0,normal
1,1554206319,9,31.75168186,0,0.312520973,0,59.14459419,31.75168186,66.24773489,427.0412837,...,0.157564294,0.189279353,3.17150584,1497232.743,355.1343322,11153.55449,15.75642941,0.000532995,0,normal
2,1554206329,5,29.49516707,0,1.16822183,0,46.02794011,29.49516707,66.90038148,1159.426821,...,0.401773775,0.782471877,38.06981014,1854228.536,760.4317685,26994.26595,40.17737752,0.001028323,0,normal
3,1554206339,12,18.22437505,0,1.097191902,0,47.80621859,18.22437505,79.54520811,736.622011,...,0.259062331,0.768934004,50.98716725,15912390.29,699.4999538,53141.6638,25.90623311,0.001099429,0,normal
4,1554206349,12,14.86118688,0,1.562431019,0,41.8731513,14.86118688,82.73513724,896.1085985,...,0.103836186,1.081729712,97.7893526,23609299.7,1184.550425,40988.68444,10.38361856,0.000912738,0,normal


In [3]:
# Attack counts by type:
pd.crosstab(index=df['label'], columns=df['type'])

type,ddos,dos,injection,mitm,normal,password,scanning,xss
label,,,,,,,,
0,0,0,0,0,24871,0,0,0
1,4608,525,612,15,0,3628,447,1269


In [4]:
# Remove the label column:
df = df.drop(['label','ts'],axis=1)

In [5]:
df.head()

,Processor_DPC_Rate,Processor_pct_ Idle_Time,Processor_pct_ C3_Time,Processor_pct_ Interrupt_Time,Processor_pct_ C2_Time,Processor_pct_ User_Time,Processor_pct_ C1_Time,Processor_pct_ Processor_Time,Processor_C1_ransitions_sec,Processor_pct_ DPC_Time,...,LogicalDisk(_Total) Avg Disk Bytes Read,LogicalDisk(_Total) Avg Disk Write Queue Length,LogicalDisk(_Total) Avg Disk Queue Length,LogicalDisk(_Total) pct_ Disk Read Time,LogicalDisk(_Total) Disk Write Bytes sec,LogicalDisk(_Total) Disk Transfers sec,LogicalDisk(_Total) Avg Disk Bytes Transfer,LogicalDisk(_Total) pct_ Disk Write Time,LogicalDisk(_Total) Avg Disk sec Transfer,type
0,4,29.90817156,0,0.078240397,0,61.02750947,29.90817156,66.2001486,480.0949912,0.078240397,...,20546.65089,0.134876119,0.163098103,2.822198461,1924988.236,402.5827836,9100.481592,13.48761188,0.000405118,normal
1,9,31.75168186,0,0.312520973,0,59.14459419,31.75168186,66.24773489,427.0412837,0.078130243,...,25742.31139,0.157564294,0.189279353,3.17150584,1497232.743,355.1343322,11153.55449,15.75642941,0.000532995,normal
2,5,29.49516707,0,1.16822183,0,46.02794011,29.49516707,66.90038148,1159.426821,0.233644366,...,35969.10535,0.401773775,0.782471877,38.06981014,1854228.536,760.4317685,26994.26595,40.17737752,0.001028323,normal
3,12,18.22437505,0,1.097191902,0,47.80621859,18.22437505,79.54520811,736.622011,0.313483401,...,40019.44778,0.259062331,0.768934004,50.98716725,15912390.29,699.4999538,53141.6638,25.90623311,0.001099429,normal
4,12,14.86118688,0,1.562431019,0,41.8731513,14.86118688,82.73513724,896.1085985,0.703093958,...,32025.59015,0.103836186,1.081729712,97.7893526,23609299.7,1184.550425,40988.68444,10.38361856,0.000912738,normal


## Checking column types and converting them in the right format:

In [6]:
print(df.info())
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35975 entries, 0 to 35974
Columns: 125 entries, Processor_DPC_Rate to type
dtypes: float64(5), int64(50), object(70)
memory usage: 34.3+ MB
None
(35975, 125)


So, there a total of 125 features presented in this data file and number of rows are 35,975!

In [7]:
columns = df.applymap(np.isreal).all()
print((columns).value_counts())

False    70
True     55
dtype: int64


So, there are a total of 70 categorical and 57 numerical columns available in this data file. We need to investigate that, the 70 categorical variables whether they are really categorical or there are some problem in data types for them. 

In [8]:
# Now considering blank space as missing entry and recategorizing the numerical object columns to actual number columns
df.replace(r'^\s*$', np.nan, regex=True)

,Processor_DPC_Rate,Processor_pct_ Idle_Time,Processor_pct_ C3_Time,Processor_pct_ Interrupt_Time,Processor_pct_ C2_Time,Processor_pct_ User_Time,Processor_pct_ C1_Time,Processor_pct_ Processor_Time,Processor_C1_ransitions_sec,Processor_pct_ DPC_Time,...,LogicalDisk(_Total) Avg Disk Bytes Read,LogicalDisk(_Total) Avg Disk Write Queue Length,LogicalDisk(_Total) Avg Disk Queue Length,LogicalDisk(_Total) pct_ Disk Read Time,LogicalDisk(_Total) Disk Write Bytes sec,LogicalDisk(_Total) Disk Transfers sec,LogicalDisk(_Total) Avg Disk Bytes Transfer,LogicalDisk(_Total) pct_ Disk Write Time,LogicalDisk(_Total) Avg Disk sec Transfer,type
0,4,29.90817156,0,0.078240397,0,61.02750947,29.90817156,66.2001486,480.0949912,0.078240397,...,20546.65089,0.134876119,0.163098103,2.822198461,1924988.236,402.5827836,9100.481592,13.48761188,0.000405118,normal
1,9,31.75168186,0,0.312520973,0,59.14459419,31.75168186,66.24773489,427.0412837,0.078130243,...,25742.31139,0.157564294,0.189279353,3.17150584,1497232.743,355.1343322,11153.55449,15.75642941,0.000532995,normal
2,5,29.49516707,0,1.16822183,0,46.02794011,29.49516707,66.90038148,1159.426821,0.233644366,...,35969.10535,0.401773775,0.782471877,38.06981014,1854228.536,760.4317685,26994.26595,40.17737752,0.001028323,normal
3,12,18.22437505,0,1.097191902,0,47.80621859,18.22437505,79.54520811,736.622011,0.313483401,...,40019.44778,0.259062331,0.768934004,50.98716725,15912390.29,699.4999538,53141.6638,25.90623311,0.001099429,normal
4,12,14.86118688,0,1.562431019,0,41.8731513,14.86118688,82.73513724,896.1085985,0.703093958,...,32025.59015,0.103836186,1.081729712,97.7893526,23609299.7,1184.550425,40988.68444,10.38361856,0.000912738,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35970,0,65.61119892,0,0.078247354,0,29.96873651,65.61119892,33.17675984,226.0873623,0,...,0,0.000486421,0.000486421,0,8872.802038,1.70291766,5210.352941,0.016214354,0.000285682,normal
35971,0,96.82941756,0,0,0,0.234245076,96.82941756,0.367761021,284.841927,0,...,65536,0.000350316,0.000431441,0.002703501,12844.07218,1.199334429,16170.66667,0.011676524,0.000359733,normal
35972,0,95.29471453,0,0,0,2.108141316,95.29471453,2.088547782,263.6426844,0,...,0,0.000167812,0.000167812,0,9824.537698,0.499701828,19660.8,0.005593727,0.00033582,normal
35973,0,94.92761932,0,0,0,0.859392566,94.92761932,2.810513447,342.1107902,0,...,0,0.000440129,0.000440129,0,21299.8718,1.500047311,14199.46667,0.0146713,0.000293413,normal


In [9]:
# Keeping the object columns only to convert them as numberical because they actualy is a numerical column but
# because of space available in the entry python considering this column as an object type.

object_colmns = list(df.select_dtypes(include=['object']).columns)
object_colmns.remove('type')

In [10]:
df[object_colmns] = df[object_colmns].apply(pd.to_numeric, errors='coerce')

In [11]:
# Checking:
columns = df.applymap(np.isreal).all()
print((columns).value_counts())

True     124
False      1
dtype: int64


Now, only the response variable seems categorical. So, we are good about our actual column types for the dataframe.

## Summary statistics for numerical columns

In [116]:
df.describe()

,Processor_DPC_Rate,Processor_pct_ Idle_Time,Processor_pct_ C3_Time,Processor_pct_ Interrupt_Time,Processor_pct_ C2_Time,Processor_pct_ User_Time,Processor_pct_ C1_Time,Processor_pct_ Processor_Time,Processor_C1_ransitions_sec,Processor_pct_ DPC_Time,...,LogicalDisk(_Total) pct_ Disk Time,LogicalDisk(_Total) Avg Disk Bytes Read,LogicalDisk(_Total) Avg Disk Write Queue Length,LogicalDisk(_Total) Avg Disk Queue Length,LogicalDisk(_Total) pct_ Disk Read Time,LogicalDisk(_Total) Disk Write Bytes sec,LogicalDisk(_Total) Disk Transfers sec,LogicalDisk(_Total) Avg Disk Bytes Transfer,LogicalDisk(_Total) pct_ Disk Write Time,LogicalDisk(_Total) Avg Disk sec Transfer
count,35975.000000,35964.000000,35964.0,35964.000000,35964.0,35964.000000,35964.000000,35964.000000,35964.000000,35964.000000,...,35964.000000,35964.000000,35964.000000,35964.000000,35964.000000,3.596400e+04,35964.000000,35964.000000,35964.000000,35964.000000
mean,5.675969,71.931669,0.0,0.169123,0.0,17.146844,71.931669,25.524752,553.755621,0.512173,...,6.050947,14733.358434,0.020836,0.139145,4.441090,1.194373e+06,235.076224,25449.075247,1.609857,0.000645
std,18.675289,20.668511,0.0,0.361975,0.0,14.045875,20.668511,20.466752,848.019303,1.229756,...,19.179710,23292.794082,0.102579,0.420395,14.663319,3.378213e+06,720.608157,48989.625464,10.021435,0.000950
min,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,0.000000,62.453948,0.0,0.000000,0.0,5.320278,62.453948,8.440140,154.861333,0.000000,...,0.014109,0.000000,0.000363,0.000422,0.000000,1.148637e+04,0.898632,11655.331267,0.012168,0.000308
50%,0.000000,74.910917,0.0,0.000000,0.0,14.531253,74.910917,22.965543,307.312866,0.078124,...,0.040476,4352.000000,0.000812,0.001201,0.001470,2.775862e+04,2.695918,16405.169650,0.027414,0.000417
75%,1.000000,89.084994,0.0,0.234015,0.0,25.194464,89.084994,34.296272,423.242618,0.390012,...,0.260759,23081.187655,0.004444,0.007436,0.057356,2.054620e+05,17.073693,24576.000000,0.151185,0.000657
max,195.000000,99.638956,0.0,10.641964,0.0,91.566420,99.638956,100.000000,10318.521390,21.953888,...,313.419250,944383.062300,1.836753,4.364882,294.529448,6.393745e+07,7170.887411,977715.200000,183.675284,0.045244


We are observing that the range of the columns are highly fluctuating for the columns. So, definitely the data needs to be standardize or we have to normalize the columns before feeding them in the model. But before that, we are going to observe the missing value distribution over the columns.

## Missing value % investigation and imputation:

In [12]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True)

In [13]:
missing_value_df.tail()

,column_name,percent_missing
Process_IO Data Bytes_sec,Process_IO Data Bytes_sec,0.678249
Process_IO Other Operations_sec,Process_IO Other Operations_sec,0.683808
Process_IO Data Operations_sec,Process_IO Data Operations_sec,0.920083
Process_IO Read_Operations_sec,Process_IO Read_Operations_sec,1.164698
Process_IO Read Bytes_sec,Process_IO Read Bytes_sec,1.495483


The highest missing percentage is around 1.5% for Process_IO Read Bytes_sec column. So, we are good to go for missing value imputation. We will replace that missing entries by mean at this stage. Later we will check the performance after replacing with Median. 

In [14]:
df.fillna(df.select_dtypes(include='number').mean(), inplace=True)

In [15]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True)

In [16]:
missing_value_df.tail()

,column_name,percent_missing
Process_ID Process,Process_ID Process,0.0
Process_Page Faults_sec,Process_Page Faults_sec,0.0
Process_Working Set,Process_Working Set,0.0
Network_I(Intel R _82574L_GNC) Bytes Received sec,Network_I(Intel R _82574L_GNC) Bytes Received sec,0.0
type,type,0.0


## Seperating independent and dependent variables:

In [17]:
# Removing Y and Id
X = df.iloc [: ,:-1].values    # independent variables


# Keeping only Y
y = df.iloc [: , -1].values    # dependent variables

## Handling Inf values:


In [18]:
import numpy as np

# find min and max values for each column, ignoring nan, -inf, and inf
mins = [np.nanmin(X[:, i][X[:, i] != -np.inf]) for i in range(X.shape[1])]
maxs = [np.nanmax(X[:, i][X[:, i] != np.inf]) for i in range(X.shape[1])]

# go through matrix one column at a time and replace  + and -infinity 
# with the max or min for that column
for i in range(X.shape[1]):
    X[:, i][X[:, i] == -np.inf] = mins[i]
    X[:, i][X[:, i] == np.inf] = maxs[i]

## Standardizing all numerical columns 

In [19]:
# feature Scaling
sc = MinMaxScaler(feature_range = (0,1))
X = sc.fit_transform(X)

# Unsupervised modelling:

## K-means clustering 

In [54]:
#Now let's open it with pandas
import pandas as pd
from pandas import Series,DataFrame
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

import numpy as np
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as shc
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

import warnings
warnings.filterwarnings('ignore')

In [55]:
data = pd.DataFrame(X)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,114,115,116,117,118,119,120,121,122,123
0,0.020513,0.300165,0.0,0.007352,0.0,0.666484,0.300165,0.662001,0.046527,0.003564,...,0.052038,0.021757,0.073432,0.037366,0.009582,0.030107,0.056141,0.009308,0.073432,0.008954
1,0.046154,0.318667,0.0,0.029367,0.0,0.645920,0.318667,0.662477,0.041386,0.003559,...,0.060392,0.027258,0.085784,0.043364,0.010768,0.023417,0.049524,0.011408,0.085784,0.011780
2,0.025641,0.296020,0.0,0.109775,0.0,0.502673,0.296020,0.669004,0.112364,0.010643,...,0.249657,0.038087,0.218741,0.179265,0.129256,0.029001,0.106044,0.027610,0.218741,0.022728
3,0.061538,0.182904,0.0,0.103101,0.0,0.522093,0.182904,0.795452,0.071388,0.014279,...,0.245337,0.042376,0.141044,0.176164,0.173114,0.248874,0.097547,0.054353,0.141044,0.024300
4,0.061538,0.149150,0.0,0.146818,0.0,0.457298,0.149150,0.827351,0.086845,0.032026,...,0.345138,0.033912,0.056532,0.247826,0.332019,0.369256,0.165189,0.041923,0.056532,0.020174


In [56]:
# from sklearn.cluster import KMeans 
clusters = 8
number_of_iteration = 50
initial_centroid_method = 'k-means++' # otherwise 'random'
  
kmeans = KMeans(n_clusters = clusters,n_init= number_of_iteration,init = initial_centroid_method) 
kmeans.fit(data) 

labels = kmeans.labels_

In [57]:
#Glue back to original data
data['clusters'] = labels

In [58]:
data["Actual_cat"] = y

In [59]:
df.columns[:-1]

Index(['Processor_DPC_Rate', 'Processor_pct_ Idle_Time',
       'Processor_pct_ C3_Time', 'Processor_pct_ Interrupt_Time',
       'Processor_pct_ C2_Time', 'Processor_pct_ User_Time',
       'Processor_pct_ C1_Time', 'Processor_pct_ Processor_Time',
       'Processor_C1_ransitions_sec', 'Processor_pct_ DPC_Time',
       ...
       'LogicalDisk(_Total) pct_ Disk Time',
       'LogicalDisk(_Total) Avg  Disk Bytes Read',
       'LogicalDisk(_Total) Avg  Disk Write Queue Length',
       'LogicalDisk(_Total) Avg  Disk Queue Length',
       'LogicalDisk(_Total) pct_ Disk Read Time',
       'LogicalDisk(_Total) Disk Write Bytes sec',
       'LogicalDisk(_Total) Disk Transfers sec',
       'LogicalDisk(_Total) Avg  Disk Bytes Transfer',
       'LogicalDisk(_Total) pct_ Disk Write Time',
       'LogicalDisk(_Total) Avg  Disk sec Transfer'],
      dtype='object', length=124)

In [60]:
data.columns = ['Processor_DPC_Rate',
  'Processor_pct_ Idle_Time',
  'Processor_pct_ C3_Time',
  'Processor_pct_ Interrupt_Time',
  'Processor_pct_ C2_Time',
  'Processor_pct_ User_Time',
  'Processor_pct_ C1_Time',
  'Processor_pct_ Processor_Time',
  'Processor_C1_ransitions_sec',
  'Processor_pct_ DPC_Time',
  'Processor_C2_ransitions_sec',
  'Processor_pct_ Privileged_Time',
  'Processor_C3_ransitions_sec',
  'Processor_DPCs_Queued_sec',
  'Processor_Interrupts_sec',
  'Process_Pool_Paged Bytes',
  'Process_IO Read_Operations_sec',
  'Process_Working_Set_ Private',
  'Process_Working_Set_Peak',
  'Process_IO_Write Operations_sec',
  'Process_Page_File Bytes',
  'Process_pct_ User_Time',
  'Process_Virtual_Bytes Peak',
  'Process_Page_File Bytes Peak',
  'Process_IO_Other_Bytes_sec',
  'Process_Private_Bytes',
  'Process_IO_Write_Bytes_sec',
  'Process_Elapsed_Time',
  'Process_Virtual_Bytes',
  'Process_pct_ Processor_Time',
  'Process_Creating Process ID',
  'Process_Pool Nonpaged Bytes',
  'Process_Working Set',
  'Process_Page Faults_sec',
  'Process_ID Process',
  'Process_IO Other Operations_sec',
  'Process_IO Data Operations_sec',
  'Process_Thread Count',
  'Process_pct_ Privileged_Time',
  'Process_IO Data Bytes_sec',
  'Process_IO Read Bytes_sec',
  'Process_Priority Base',
  'Process_Handle Count',
  'Network_I(Intel R _82574L_GNC)TCP_APS',
  'Network_I(Intel R _82574L_GNC) Packets Received Unknown',
  'Network_I(Intel R _82574L_GNC) Bytes Received sec',
  'Network_I(Intel R _82574L_GNC) Bytes Sent sec',
  'Network_I(Intel R _82574L_GNC) Packets Outbound Errors',
  'Network_I(Intel R _82574L_GNC) Packets Received Discarded',
  'Network_I(Intel R _82574L_GNC) Bytes Total sec',
  'Network_I(Intel R _82574L_GNC) Packets Outbound Discarded',
  'Network_I(Intel R _82574L_GNC) TCP RSC Exceptions sec',
  'Network_I(Intel R _82574L_GNC) Packets Sent Unicast sec',
  'Network_I(Intel R _82574L_GNC) Output Queue Length',
  'Network_I(Intel R _82574L_GNC) Packets Received sec',
  'Network_I(Intel R _82574L_GNC) Current Bandwidth',
  'Network_I(Intel R _82574L_GNC) Packets sec',
  'Network_I(Intel R _82574L_GNC) TCP Active RSC Connections',
  'Network_I(Intel R _82574L_GNC) Packets Sent sec',
  'Network_I(Intel R _82574L_GNC) Packets Received Unicast sec',
  'Network_I(Intel R _82574L_GNC) Packets Sent Non-Unicast sec',
  'Network_I(Intel R _82574L_GNC) Packets Received Non-Unicast sec',
  'Network_I(Intel R _82574L_GNC) TCP RSC Coalesced Packets sec',
  'Network_I(Intel R _82574L_GNC) Offloaded Connections',
  'Network_I(Intel R _82574L_GNC) Packets Received Errors',
  'Memory Pool Paged Bytes',
  'Memory Free & Zero Page List Bytes',
  'Memory Cache Bytes Peak',
  'Memory System Code Resident Bytes',
  'Memory Available Bytes',
  'Memory Commit Limit',
  'Memory Transition Pages RePurposed sec',
  'Memory Pages Output sec',
  'Memory Page Reads sec',
  'Memory Demand Zero Faults sec',
  'Memory Available KBytes',
  'Memory Pages sec',
  'Memory Cache Bytes',
  'Memory Pool Nonpaged Bytes',
  'Memory Page Faults sec',
  'Memory Transition Faults sec',
  'Memory System Cache Resident Bytes',
  'Memory Long-Term Average Standby Cache Lifetime (s)',
  'Memory Standby Cache Reserve Bytes',
  'Memory Page Writes sec',
  'Memory System Code Total Bytes',
  'Memory Standby Cache Core Bytes',
  'Memory System Driver Resident Bytes',
  'Memory Standby Cache Normal Priority Bytes',
  'Memory Pool Paged Allocs',
  'Memory Pool Nonpaged Allocs',
  'Memory pct_ Committed Bytes In Use',
  'Memory Free System Page Table Entries',
  'Memory Available MBytes',
  'Memory Modified Page List Bytes',
  'Memory Cache Faults sec',
  'Memory Committed Bytes',
  'Memory System Driver Total Bytes',
  'Memory Pages Input sec',
  'Memory Pool Paged Resident Bytes',
  'Memory Write Copies sec',
  'LogicalDisk(_Total) Avg  Disk Bytes Write',
  'LogicalDisk(_Total) pct_ Idle Time',
  'LogicalDisk(_Total) Disk Reads sec',
  'LogicalDisk(_Total) pct_ Free Space',
  'LogicalDisk(_Total) Disk Read Bytes sec',
  'LogicalDisk(_Total) Avg  Disk sec Read',
  'LogicalDisk(_Total) Disk Writes sec',
  'LogicalDisk(_Total) Current Disk Queue Length',
  'LogicalDisk(_Total) Split IO Sec',
  'LogicalDisk(_Total) Free Megabytes',
  'LogicalDisk(_Total) Avg  Disk sec Write',
  'LogicalDisk(_Total) Disk Bytes sec',
  'LogicalDisk(_Total) Avg  Disk Read Queue Length',
  'LogicalDisk(_Total) pct_ Disk Time',
  'LogicalDisk(_Total) Avg  Disk Bytes Read',
  'LogicalDisk(_Total) Avg  Disk Write Queue Length',
  'LogicalDisk(_Total) Avg  Disk Queue Length',
  'LogicalDisk(_Total) pct_ Disk Read Time',
  'LogicalDisk(_Total) Disk Write Bytes sec',
  'LogicalDisk(_Total) Disk Transfers sec',
  'LogicalDisk(_Total) Avg  Disk Bytes Transfer',
  'LogicalDisk(_Total) pct_ Disk Write Time',
  'LogicalDisk(_Total) Avg  Disk sec Transfer',
 'Predicted',
 'Actual']

In [61]:
pd.crosstab(index=data['Actual'], columns=data['Predicted']) # k-means++ (50 iteration)

Predicted,0,1,2,3,4,5,6,7
Actual,,,,,,,,
ddos,43,1,3908,0,0,656,0,0
dos,436,57,32,0,0,0,0,0
injection,607,5,0,0,0,0,0,0
mitm,1,0,14,0,0,0,0,0
normal,2016,655,1165,7473,4501,4288,1807,2966
password,298,59,3271,0,0,0,0,0
scanning,12,39,26,0,0,370,0,0
xss,0,1266,3,0,0,0,0,0


In [43]:
pd.crosstab(index=data['Actual'], columns=data['Predicted']) # k-means++

Predicted,0,1,2,3,4,5,6,7
Actual,,,,,,,,
ddos,18,0,4410,0,179,0,0,1
dos,0,0,23,0,438,0,0,64
injection,0,0,0,0,605,0,0,7
mitm,0,0,14,0,1,0,0,0
normal,4175,4501,1202,7473,2069,1807,2966,678
password,0,0,3260,0,300,0,0,68
scanning,359,0,35,0,12,0,0,41
xss,0,0,3,0,0,0,0,1266


In [51]:
pd.crosstab(index=data['Actual'], columns=data['Predicted']) # random (50 iteration)

Predicted,0,1,2,3,4,5,6,7
Actual,,,,,,,,
ddos,0,656,0,3908,43,0,1,0
dos,0,0,0,32,436,0,57,0
injection,0,0,0,0,607,0,5,0
mitm,0,0,0,14,1,0,0,0
normal,7473,4288,2966,1165,2016,1807,655,4501
password,0,0,0,3271,298,0,59,0
scanning,0,370,0,26,12,0,39,0
xss,0,0,0,3,0,0,1266,0


In [35]:
pd.crosstab(index=data['Actual'], columns=data['Predicted']) # random

Predicted,0,1,2,3,4,5,6,7
Actual,,,,,,,,
ddos,654,0,0,1,43,3910,0,0
dos,0,0,0,57,436,32,0,0
injection,0,0,0,5,607,0,0,0
mitm,0,0,0,0,1,14,0,0
normal,4287,7471,2966,655,2016,1166,4501,1809
password,0,0,0,59,298,3271,0,0
scanning,370,0,0,39,12,26,0,0
xss,0,0,0,1266,0,3,0,0


## Use K-means clusterign for two category: